# Bayesian analysis and MCMC

## Defining priors

In [1]:
from likelihood import JointPriors

# Defining the priors and the type of distribuition
param_configs_w_frb = {
    'H0': ((20, 100), 'uniform'),
    'Omega_bh2': ((0, 0.05), 'uniform'), 
    'Omega_m': ((0, 0.5), 'uniform'),
    'DM_host_0': ((55, 225), 'uniform'),
    'omega_0': ((-3, 0), 'uniform')
}

param_configs_CPL_frb = {
    'H0': ((20, 100), 'uniform'),
    'Omega_bh2': ((0, 0.05), 'uniform'), 
    'Omega_m': ((0, 0.5), 'uniform'),
    'DM_host_0': ((55, 225), 'uniform'),
    'omega_0': ((-3, 0), 'uniform'),
    'omega_a': ((-3, 3), 'uniform')
}

param_configs_w_sne = {
    'H0': ((20, 100), 'uniform'),
    'Omega_m': ((0, 0.5), 'uniform'),
    'omega_0': ((-3, 0), 'uniform')
}

param_configs_CPL_sne = {
    'H0': ((20, 100), 'uniform'),
    'Omega_m': ((0, 0.5), 'uniform'),
    'omega_0': ((-3, 0), 'uniform'),
    'omega_a': ((-3, 3), 'uniform')
}

P_w_frb = JointPriors(param_configs_w_frb)
P_w_sne = JointPriors(param_configs_w_sne)
P_CPL_frb = JointPriors(param_configs_CPL_frb)
P_CPL_sne = JointPriors(param_configs_CPL_sne)

### Models and parameters

In [ ]:
from likelihood import JointLikelihoodFunction
from equations import DM_EXT_model, Modulus_sne

model = DM_EXT_model()
mu_wCDM = Modulus_sne()

# Creating an instance of JointLikelihoodFunction
LF_w_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2,  
        Omega_m=Omega_m,
        H0=H0,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,  
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    }
)

LF_w_sne = JointLikelihoodFunction(
    { 'SNe': lambda z, H0, Omega_m, omega_0: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H0,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    }
)

LF_w_sne_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2, 
        H0=H0, 
        Omega_m=Omega_m,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,  
        cosmo_type='non_standard',
        param_type='constant'
    ),

    'SNe': lambda z, H0, Omega_m, omega_0: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H0,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant'
    ) 
    }
)

LF_CPL_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0, omega_a: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2, 
        H0=H0, 
        Omega_m=Omega_m,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,
        omega_a=omega_a,  
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    }
)

LF_CPL_sne = JointLikelihoodFunction(
    { 'SNe': lambda z, H0, Omega_m, omega_0, omega_a: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H0,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    }
)

LF_CPL_sne_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, H0, Omega_m, DM_host_0, omega_0, omega_a: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2,  
        H0=H0,
        Omega_m=Omega_m,     
        DM_host_0=DM_host_0,
        omega_0=omega_0,
        omega_a=omega_a,  
        cosmo_type='non_standard',
        param_type='CPL'
    ),

    'SNe': lambda z, H0, Omega_m, omega_0, omega_a: mu_wCDM.Modulo_std(
        z=z,
        Omega_m=Omega_m,
        H_0=H0,
        omega_0=omega_0,
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'
    ) 
    }
)

## Preparing the samples

### Analysis for 66 FRBs

In [ ]:
from obs_data import FRB_data, SNe_data
import ultranest

# Instantiate the FRB_data class for 66 FRBs
frb_data = FRB_data(n_frb=66)

sne_data = SNe_data(sample_sne='Pantheon+')

z_sne, mu_sne, cov_matrix = sne_data.load_data()

# Call the select_data method to get the observed data
z_frb, DM_ext, sigma_ext = frb_data.select_data()

# Configuring the ultranest samplers
sampler_w_frb = ultranest.ReactiveNestedSampler(
    P_w_frb.param_names,
    lambda params: LF_w_frb.log_likelihood(
        dict(zip(P_w_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext)
        }
    ),
    P_w_frb.prior_transform
)

sampler_w_sne = ultranest.ReactiveNestedSampler(
    P_w_sne.param_names,
    lambda params: LF_w_sne.log_likelihood(
        dict(zip(P_w_sne.param_names, params)),
        {
            'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_w_sne.prior_transform
)

sampler_w_sne_frb = ultranest.ReactiveNestedSampler(
    P_w_frb.param_names,
    lambda params: LF_w_sne_frb.log_likelihood(
        dict(zip(P_w_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext),
            'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_w_frb.prior_transform
)

sampler_CPL_frb = ultranest.ReactiveNestedSampler(
    P_CPL_frb.param_names,
    lambda params: LF_CPL_frb.log_likelihood(
        dict(zip(P_CPL_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext)
        }
    ),
    P_CPL_frb.prior_transform
)

sampler_CPL_sne = ultranest.ReactiveNestedSampler(
    P_CPL_sne.param_names,
    lambda params: LF_CPL_sne.log_likelihood(
        dict(zip(P_CPL_sne.param_names, params)),
        {
            'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_CPL_sne.prior_transform
)

sampler_CPL_sne_frb = ultranest.ReactiveNestedSampler(
    P_CPL_frb.param_names,
    lambda params: LF_CPL_sne_frb.log_likelihood(
        dict(zip(P_CPL_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext),
            'SNe': (z_sne, mu_sne, cov_matrix)
        }
    ),
    P_CPL_frb.prior_transform
)

In [ ]:
wCDM_frb = sampler_w_frb.run(min_num_live_points=400)
sampler_w_frb.print_results()

In [ ]:
wCDM_sne = sampler_w_sne.run(min_num_live_points=400)
sampler_w_sne.print_results()

In [ ]:
wCDM_sne_frb = sampler_w_sne_frb.run(min_num_live_points=400)
sampler_w_sne_frb.print_results()

In [ ]:
CPL_frb = sampler_CPL_frb.run(min_num_live_points=400)
sampler_CPL_frb.print_results()

In [ ]:
CPL_sne = sampler_CPL_sne.run(min_num_live_points=400)
sampler_CPL_sne.print_results()

In [ ]:
CPL_sne_frb = sampler_CPL_sne_frb.run(min_num_live_points=400)
sampler_CPL_sne_frb.print_results()

In [ ]:
from getdist import plots, MCSamples

# Extraindo amostras dos resultados para frb
samples1_frb = wCDM_frb['samples']
samples2_frb = CPL_frb['samples']

labels1 = ['H_0', '\\Omega_{b}h^2', '\\Omega_m', 'DM_{host,0}', '\\omega_0']
names1 = P_w_frb.param_names
mcsamples1_frb = MCSamples(samples=samples1_frb, names=names1, labels=labels1)

labels2 = ['H_0', '\\Omega_{b}h^2', '\\Omega_m', 'DM_{host,0}', '\\omega_0', '\\omega_a']
names2 = P_CPL_frb.param_names
mcsamples2_frb = MCSamples(samples=samples2_frb, names=names2, labels=labels2)

# Extraindo amostras dos resultados para sne
samples1_sne = wCDM_sne['samples']
samples2_sne = CPL_sne['samples']

labels3 = ['H_0', '\\Omega_m', '\\omega_0']
names3 = P_w_sne.param_names
mcsamples1_sne = MCSamples(samples=samples1_sne, names=names3, labels=labels3)

labels4 = ['H_0', '\\Omega_m', '\\omega_0', '\\omega_a']
names4 = P_CPL_sne.param_names
mcsamples2_sne = MCSamples(samples=samples2_sne, names=names4, labels=labels4)

# Extraindo amostras dos resultados para sne+frb

samples1_sne_frb = wCDM_sne_frb['samples']
samples2_sne_frb = CPL_sne_frb['samples']

mcsamples1_sne_frb = MCSamples(samples=samples1_sne_frb, names=names1, labels=labels1)
mcsamples2_sne_frb = MCSamples(samples=samples2_sne_frb, names=names2, labels=labels2)



In [ ]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples1_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples2_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples1_frb, mcsamples2_frb], filled=True, contour_colors=['green', 'red'], 
                legend_labels=['$\omega_0$CDM with FRB', '$\omega_0\omega_a$CDM with FRB'])
g.export('Figuras/wCDM_CPL_frb.png', dpi=600)

In [ ]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples1_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples1_sne.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples1_sne_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples1_frb, mcsamples1_sne, mcsamples1_sne_frb], 
                params=['H_0', 'Omega_m', 'omega_0', 'omega_a'] , filled=True, 
                contour_colors=['green', 'red', 'blue'], 
                legend_labels=['$\omega_0$CDM with FRB', '$\omega_0$CDM with SNe', '$\omega_0$CDM with SNe + FRB'])
g.export('Figuras/param_constant_cosmo.png', dpi=600)

In [ ]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples2_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples2_sne.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples2_sne_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples2_frb, mcsamples2_sne, mcsamples2_sne_frb], 
                params=['H_0', 'Omega_m', 'omega_0', 'omega_a'] , filled=True, 
                contour_colors=['green', 'red', 'blue'], 
                legend_labels=['$\omega_0\omega_a$CDM with FRB', '$\omega_0\omega_a$CDM with SNe', '$\omega_0\omega_a$CDM with SNe + FRB'])
g.export('Figuras/param_CPL_cosmo.png', dpi=600)

## Model Comparison

In [17]:
samples = {
    "Sample_frb": {
        "results": {
            'wCDM': {'logz': wCDM_frb['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_frb['logz'], 'num_params': 6},
        }
    },
    "Sample_sne": {
        "results": {
            'wCDM': {'logz': wCDM_sne['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_sne['logz'], 'num_params': 6},
        }
    },
    "Sample_sne_frb": {
        "results": {
            'wCDM': {'logz': wCDM_sne_frb['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_sne_frb['logz'], 'num_params': 6},
        }
    }
}

In [18]:
from bayesian_analysis import ModelComparison

comparison = ModelComparison(samples)
comparison.run_comparisons(save_to_file='comparisons_output_frb+SNe.txt')

In [19]:
from bayesian_analysis import SaveResults

# Initialize the class with the desired output file
results_saver = SaveResults("model_results.txt")

# Optional: Clear the file if you want to start fresh
results_saver.reset_file()

results_saver.save_to_txt(mcsamples1_frb, 'wCDM_frb')
results_saver.save_to_txt(mcsamples2_frb, 'CPL_frb')
results_saver.save_to_txt(mcsamples1_sne, 'wCDM_sne')
results_saver.save_to_txt(mcsamples2_sne, 'CPL_sne')
results_saver.save_to_txt(mcsamples1_sne_frb, 'wCDM_sne_frb')
results_saver.save_to_txt(mcsamples2_sne_frb, 'CPL_sne_frb')


In [ ]:
"""from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_16, y=dm_ext_obs_16, fmt='o', alpha=0.6, color='red', label='16 FRBs', ms=2)

z_test = np.linspace(0, 1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0, omega_a  in sampler_p2_16.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,  
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'))

band.line(color='k', linestyle='-', label='CPL parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_16.png', format='png', dpi=600)"""

In [ ]:
"""from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_63, y=dm_ext_obs_63, fmt='o', alpha=0.6, color='red', label='63 FRBs', ms=2)

z_test = np.linspace(0, 1.1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0  in sampler_constant_63.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant')
    )

band.line(color='k', linestyle='-', label='Constant parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_63.png', format='png', dpi=600)"""